<img src="../figures/header.png" alt="Header" width="450"/>

En los últimos meses, los LLMs (Large Language Models) han demostrado ser una herramienta sumamente útil y poderosa para un sin fin de tareas. De hecho, medio año después del lanzamiento oficial de [ChatGPT](https://openai.com/blog/chatgpt), todavía estamos rascando la superficie y descubriendo nuevos usos potenciales cada semana. Pero no todo es color de rosas, estos modelos todavía no son lo suficientemente desarrollados para "leer nuestras mentes". Por esto, necesitan recibir instrucciones claras y libres de ambigüedades, para ejecutar tareas de forma exitosa. En pocas palabras, si queremos liberar el verdadero potencial de los LLMs necesitamos aprender a hablar su idioma.


**¿Qué vamos a ver en este Notebook?**

En este Notebook, vamos a experimentar dos técnicas muy poderosas que cuando se utilizan en conjunto, logran obtener resultados realmente precisos: Chain-of-Thought (CoT) y Self-Consistency.

**¿Qué herramientas vamos a utilizar?**

Por simplicidad y afinidad a la implementación actual de los productos de Cognitive Solutions, para los ejemplos de prompts vamos a utilizar el modelo más reciente de OpenAI disponible a través de la API oficial (al momento de elaborar estos tutoriales dicho modelo era `gpt-3.5-turbo`). Además, para consumir la API de OpenAI vamos a utilizar el [sdk](https://pypi.org/project/openai/) oficial, que se encuentra disponible en pypi. Para el resto, funciones y lógica adicional para el procesamiento de las respuestas del modelo, vamos a utilizar puro Python y alguna librería sencilla para algún problema específico pero que no merece la pena mencionar todavía.

Por último, es necesario contar con una API Key válida con créditos disponibles para realizar llamadas a la API de Open AI. Para más información sobre cómo generar y administrar las API Key puedes revisar la sección [OpenAI API Key setup](#openai-api-setup).

## Índice
<a class="anchor" id="index"></a>

1. [Setups](#setups)
    1. [OpenAI API Key setup](#openai-api-setup)
    2. [Import & Utils](#imports)
2. [Chain-of-Thought Prompting](#cot)
    1. [Zero-Shot Chain of Thought](#zero-shot-cot)
    2. [Few-Shot Chain of Thought](#few-shot-cot)
3. [Self-Consistency](#self-consistency)
4. [References](#references)

## Setups
<a class="anchor" id="setups"></a>

### 1. OpenAI API Key setup
<a class="anchor" id="openai-api-setup"></a>

ToDo

Para simplificar el manejo de las API Keys y evitar que queden "harcodeadas" en este notebook, utilizamos la librería [python-dotenv](https://pypi.org/project/python-dotenv/) que permite facilmente cargar variables de entorno definidas en un archivo local. En pocas palabras, lo único que necesitamos asegurarnos es que exista un archivo de nombre `.env` en la raiź de este proyecto, con el siguiente contenido:


```yaml
OPENAI_API_KEY=[A_VALID_OPENAI_API_KEY]
```


Como parte de la sección **Import&Utils** vamos a cargar esta variable de entorno utilizando los métodos `load_dotenv` y `find_dotenv` de esta libreria y finalmente setear las clases correspondientes del sdk de OpenAI para que utlicen la API Key cargada en memoria. 

### 2. Imports & Utils
<a class="anchor" id="impots"></a>

In [4]:
import openai
import os
import pandas as pd

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')  # Be sure you have a .env file defined with your OPENAI_API_KEY 

In [5]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=1.0, top_p=1.0):
    """
    
    """
    
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        top_p=top_p
    )
    return response.choices[0].message["content"]

def save(path: str, text: str):
    """
    Helper function to save prompts and model responses as TXT files
    """
    
    with open(f"{path}.txt", "w") as text_file:
        text_file.write(text)

In [6]:
"""
Styles:

This cell is to provide CSS styles for the model badge that indicates which model was used for some 
specific prompt.
"""

from IPython.core.display import HTML
HTML("""
<style>

.label {
  color: white;
  padding: 5px;
}

.success {background-color: #04AA6D;} /* Green */
.info {background-color: #2196F3;} /* Blue */

</style>
""")

## 1. Chain-of-Thought Prompting
<a class="anchor" id="cot"></a>

🔝 [Ir al Índice](#index)

Los LLM de gran dimensión como GPT-3 o PaLM han exhibido una capacidad impresionante para la comprensión del lenguaje natural y han demostrado ser extraordinariamente efectivos en tareas como extraer información de textos y generar respuestas en un estilo humano consistente. Aun así, los LLM han demostrado ser bastante robustos al realizar tareas desconocidas cuando se incluyen algunos ejemplos como parte del prompt. Esta técnica, popularizada como few-shot prompting por [Brown et al. (2020)](https://proceedings.neurips.cc/paper/2020/file/1457c0d6bfcb4967418bfb8ac142f64a-Paper.pdf), ha probado ser muy buena en aumentar el rendimiento de modelos en varios benchmarks de referencia. Sin mencionar que ahorra dinero y tiempo al ajustar el modelo a un nuevo dominio específico.

Sin embargo, los estudios demostraron que modelos como estos disminuyen abruptamente su rendimiento en tareas comunes de razonamiento o problemas de matemáticas. A pesar de la capacidad de citar el poema épico griego La Odisea, estos modelos se trancan con problemas escolares fundamentales en lógica y matemáticas.

<img src="../figures/4.webp" alt="Principles of Prompt Engineering" />

Entonces, ¿qué opciones tenemos? ¿Enviamos GPT-3 por un par de semestres a la escuela? Afortunadamente, existe una alternativa más barata y menos embarazosa. 

De manera similar a cómo los humanos abordan problemas complejos dividiéndolos en subproblemas más simples y siguiendo una línea de razonamiento lógico, podemos instruir a un modelo de lenguaje para que haga lo mismo. Este enfoque fue explorado por Wei J et al. en [Chain-of-Thought Prompting Elicits Reasoning in Large Language Models (2022)](https://arxiv.org/abs/2201.11903). Demostró resultados impresionantes en varios benchmark de referencia, confirmando Chain-of-Thought (CoT) como un enfoque sólido para mejorar el rendimiento de LLM en tareas de razonamiento comunes.

En la imagen a continuación (tomada del artículo de Wei J), un modelo LLM se apresura a llegar a una conclusión incorrecta al calcular la cantidad de manzanas que quedan en una cafetería. Esto sucede incluso cuando se proporciona un problema de razonamiento similar usando pelotas de tenis como parte del contexto.

Sin embargo, cuando se incluye en el contexto (CoT) un proceso paso a paso para resolver el problema, el modelo puede llegar con precisión a una solución válida.

<img src="../figures/3.png" alt="Principles of Prompt Engineering" />

Utilizando CoT, modelos de lenguaje de escala suficiente (equivalente en parámetros a 100B) pueden:

1. Descomponer problemas de varios pasos en pasos intermedios, lo que significa que se pueden asignar cálculos adicionales a problemas que requieren más pasos de razonamiento.

2. Proporcionar una ventana interpretable del comportamiento del modelo, sugiriendo cómo podría haber llegado a una respuesta particular y brindando oportunidades para depurar dónde salió mal el camino del razonamiento.

3. Extender el campo de aplicación a problemas matemáticos, razonamiento de sentido común y manipulación simbólica.

Este método se puede utilizar proporcionando ejemplos de razonamiento al modelo (few-shot) o ningún ejemplo (zero-shot). Veamos ambos sabores en la práctica con un ejemplo de aplicación real.



### 1.1. Zero-Shot Chain of Thought
<a class="anchor" id="zero-shot-cot"></a>

🔝 [Ir al Índice](#index)

Imagina que estamos desarrollando una nueva aplicación de compras para Walmart, con la característica innovadora de comparar y seleccionar qué producto debe comprar en función de los precios y atributos de las alternativas de diferentes marcas.

Para ilustrar el problema, centrémonos en una lista reducida de todas las opciones en [Barrasd de Jabon](https://www.walmart.com/browse/personal-care/bar-soap/1005862_1071969_1694681_4735846?povid=GlobalNav_rWeb_PersonalCare_BathBody_BarSoap) que tiene Walmart en sus tiendas. Como ves, tenemos packs desde 1 barra hasta 14 barras y gran variedad de marcas y precios (desde opciones baratas hasta opciones caras).

**Products 🛒**

In [7]:
products = f"""
---
Dove Men+Care 8 bars pack
$ 9.99

---
Dove Beauty Bar 4 bars pack
$ 6.47

---
Dove Beauty Bar 1 bars
$ 1.47

---
Dove Beauty Bar 14 bars pains
$ 16

---
Yardley London Soap Bar (Pack of 10)
$ 19.99

---
Dr. Squatch All Natural Bar Soap for Men, 5 Bar Variety Pack
$46.45
"""

Para determinar qué opción es más conveniente, podemos calcular el precio por barra en cada alternativa (precio unitario). A continuación, seleccionar la opción más económica. Siguiendo este razonamiento, descubrimos que Dove Beauty Bar 14 barras es la opción más económica con un precio unitario de $1,14 (-22% por barra que Dove Beauty Bar 1 barras).

**Reasoning 🧠**

```
- Dove Men+Care 8 bars pack: $1,24875 / unit
- Dove Beauty Bar 4 bars pack: 1,6175 / unit
- Dove Beauty Bar 1 bars: 1,47 / unit
- Dove Beauty Bar 14 bars pains: $1,142857 per / unit
- Yardley London Soap Bar (Pack of 10): 1,999 / unit
- Dr. Squatch All Natural Bar Soap for Men, 5 Bar Variety Pack: $9,29 / unit
```

Veamos si GPT-4 es lo suficientemente inteligente como para resolver este problema con un simple prompt. Además, con el espíritu de hacer ejemplos más limpios y fáciles de leer, separaremos las instrucciones del sistema del resto de la entrada usando Python [string literals](https://docs.python.org/3/reference/lexical_analysis.html#formatted-string-literals).

**Prompt 💬**

In [8]:
prompt = f"""
You will be provided with a list of various product presentations delimited by <>. Each product presentation is accompanied by its price. Your task is to identify which is the best option based on the price.

<{products}>
"""

**Response 🤖** &nbsp;&nbsp;<span class="label success">GPT-4</span>

```python
The best option based on price is Dove Beauty Bar 1 bars for $1.47.
```

In [9]:
# Your turn
response = get_completion(prompt)
print(response)

Based on the prices provided, the best option would be the Dove Beauty Bar 4 bars pack priced at $6.47.


El modelo eligió la opción más barata en función del precio total en lugar de calcular el precio unitario e identificar una oportunidad de ahorro comprando más barras. Vale la pena señalar cierta ambigüedad en la pregunta. ¿Qué significa la mejor opción? ¿Significa la más barata o la que maximiza un ahorro de dinero a largo plazo?

Reescribamos el prompt indicando al modelo que siga una cadena de pensamiento para resolver este problema agregando las palabras mágicas _“reasoning step by step”_. Además subrayando explícitamente el objetivo de encontrar una oportunidad para ahorrar dinero.

**Prompt 💬**

In [10]:
prompt = f"""
You will be provided with a list of various product presentations delimited by <>. Each product presentation is accompanied by its price. Your task is to identify which presentation offers the best money-saving opportunity, reasoning step-by-step:

<{products}>
"""

**Response 🤖** &nbsp;&nbsp;<span class="label success">GPT-4</span>

```python
The first step is to compare price per unit for products of the same type (i.e. bars of soap). 

- Dove Men+Care 8 bars pack: $9.99 ÷ 8 bars = $1.25 per bar
- Dove Beauty Bar 4 bars pack: $6.47 ÷ 4 bars = $1.62 per bar
- Dove Beauty Bar 1 bars: $1.47 per bar
- Dove Beauty Bar 14 bars pack: $16 ÷ 14 bars = $1.14 per bar
- Yardley London Soap Bar (Pack of 10): $19.99 ÷ 10 bars = $1.99 per bar
- Dr. Squatch All Natural Bar Soap for Men, 5 Bar Variety Pack: $46.45 ÷ 5 bars = $9.29 per bar

Based on this analysis, the best money-saving opportunity is the Dove Beauty Bar 14 bars pack, which offers the lowest price per bar at $1.14.
```

In [11]:
# Now is your turn!
response = get_completion(prompt)
print(response)

To identify the best money-saving opportunity, we need to compare the prices of the different product presentations. 

The first option is the Dove Men+Care 8 bars pack, priced at $9.99.

The second option is the Dove Beauty Bar 4 bars pack, priced at $6.47.

The third option is the Dove Beauty Bar 1 bar, priced at $1.47.

The fourth option is the Dove Beauty Bar 14 bars pack, priced at $16.

The fifth option is the Yardley London Soap Bar (Pack of 10), priced at $19.99.

The sixth option is the Dr. Squatch All Natural Bar Soap for Men, 5 Bar Variety Pack, priced at $46.45.

By comparing the prices, we can see that the Dove Beauty Bar 1 bar is the cheapest option at $1.47. However, it is important to consider the number of bars included in each pack to make a fair comparison.

The Dove Men+Care 8 bars pack offers 8 bars for $9.99, making each bar cost approximately $1.25 (9.99 / 8).

The Dove Beauty Bar 4 bars pack offers 4 bars for $6.47, making each bar cost approximately $1.62 (6.47

Notar que con el nuevo prompt, el modelo da una respuesta correcta y, al mismo tiempo, tenemos un razonamiento paso a paso que es más fácil de debugear y encontrar posibles errores. Aún más, es bastante impresionante cómo agregando las palabras mágicas "reasoning step-by-step" en el prompt, se produce una diferencia significativa en la salida del modelo. ¿Cuál es el truco?

El truco es obligar al modelo a generar una cadena de razonamiento paso a paso (Chain of Thought) en lugar de simplemente generar un valor numérico o booleano (resultado final). Al adoptar esta estrategia, inicialmente guiamos el modelo hacia resultados de razonamiento lógico, lo que permite que el modelo base sus respuestas en problemas similares encontrados durante el entrenamiento. En segundo lugar, ayudamos al modelo solicitando que divida los problemas más grandes en otros más pequeños y manejables. Luego, se requiere que el modelo produzca resultados coherentes para subproblemas simples, como identificar precios y la cantidad de barras en cada paquete, calcular el precio unitario y, finalmente, realizar comparaciones. Además, debido a la forma en que los modelos de lenguaje autorregresivos como GPT-3 o GPT-4 generan secuencias token por token, donde cada nuevo token se genera en función de todos los tokens generados anteriormente, la cadena de pensamiento en sí tiene una fuerte influencia positiva. en la generación de un resultado final coherente.

En otras palabras, si, a través de un razonamiento válido, llegamos a un punto a medio camino de la solución correcta, nuestras posibilidades de llegar a esta solución son mayores que cuando empezamos a razonar sobre el problema.

### 1.2. Few-Shot Chain of Thought
<a class="anchor" id="few-shot-cot"></a>

🔝 [Ir al Índice](#index)

Mientras que en Zero-Shot Chain-of-Thought dejamos al modelo libre para elaborar una cadena de razonamiento coherente y válida que conduzca a un resultado correcto, un segundo tipo de prompting llamado Few-Shot Chain-of-Thought proporciona algunos ejemplos de un problema. junto con una respuesta de razonamiento, como parte del prompt. De esta forma, se impone al modelo una forma de razonamiento, limitando las posibles respuestas.

Volviendo a nuestra aplicación de Walmart, supongamos que queremos identificar qué presentación de refresco Coca Cola tiene la mejor relación precio por onza líquida. Para ello, incorporaremos al prompt una cadena de razonamiento similar, para encontrar las mejores presentaciones de las papas fritas Lay's.

**Prompt 💬**

In [12]:
prompt = f"""
Q: Walmart has the following presentations for Lay's Classic Potato Chips: a 13 oz bag for $4.98, an 8 oz bag for $2.76, and a pack of x10 units 1 oz each for $6.82. Which presentation has the lowest price by ounce?

A: Presentation 1 costs 4.98 so has a price of 4.98/13 = 0,383076923 per ounce. Presentation 2 costs 2.76 so has a price of 2.76/8 = 0,345 / per ounce. Presentation 3 costs 6.82 and includes 10 packages so has a price of 6.82/(1 x 10) = 0.682 per ounce. In conclusion, the cheapest presentation is the second one with a price of 0.345 per ounce.
 
Q: Walmart has the following presentation for Coca-Cola Soda:
an x24 pack of 12 fl oz cans for $11.86, an x12 pack of 12 fl oz cans for $7.46, and an x6 pack of 16.9 fl oz bottles for $4.98. Which presentation has the lowest price by fluid ounce?
"""

Se alienta al lector, a que hagas tus propios experimentos y encuentre la mejor oferta. Encontrará que el modelo está en lo correcto.

**Response 🤖** &nbsp;&nbsp;<span class="label success">GPT-4</span>

```python
A: The 24-pack contains a total of 24 x 12 = 288 fluid ounces and costs $11.86, so the price per fluid ounce is 11.86/288 = $0.041 per fluid ounce. The 12-pack contains a total of 12 x 12 = 144 fluid ounces and costs $7.46, so the price per fluid ounce is 7.46/144 = $0.052 per fluid ounce. The 6-pack contains a total of 6 x 16.9 = 101.4 fluid ounces and costs $4.98, so the price per fluid ounce is 4.98/101.4 = $0.049 per fluid ounce. Therefore, the 24-pack has the lowest price per fluid ounce.
```

Por más increíble que parezca, este sencillo truco le permite al modelo inferir una cadena de razonamiento similar y lograr resultados aún más precisos que con la técnica anterior. Además, tenga en cuenta que no estamos codificando un procedimiento paso a paso para resolver el problema. Por lo tanto, este enfoque debería ser, en teoría, lo suficientemente flexible para resolver cualquier desafío similar, basándose únicamente en las habilidades de razonamiento del modelo.

In [13]:
# Now is your turn!
response = get_completion(prompt)
print(response)

A: To determine the lowest price by fluid ounce, we need to calculate the price per fluid ounce for each presentation.

Presentation 1: 24-pack of 12 fl oz cans for $11.86
Price per fluid ounce = 11.86 / (24 x 12) = 0.163 / fl oz

Presentation 2: 12-pack of 12 fl oz cans for $7.46
Price per fluid ounce = 7.46 / (12 x 12) = 0.052 / fl oz

Presentation 3: 6-pack of 16.9 fl oz bottles for $4.98
Price per fluid ounce = 4.98 / (6 x 16.9) = 0.049 / fl oz

Based on the calculations, Presentation 3 has the lowest price per fluid ounce at $0.049.


## 2. Self-Consistency
<a class="anchor" id="self-consistency"></a>

🔝 [Ir al Índice](#index)

Hay algunos escenarios en los que un LLM puede no generar respuestas consistentes, incluso usando CoT y para el mismo prompt, es posible que el modelo se confunda entre ejecuciones y devuelva una respuesta inconsistente. En este contexto, [Self-Consistency](https://arxiv.org/abs/2203.11171) es un enfoque simple pero efectivo, que consiste simplemente en preguntar al modelo con el mismo prompt varias veces y tomar el resultado de la mayoría, como respuesta final. La combinación de esta técnica con CoT ayuda a obtener resultados más robustos y predecibles.

Por ejemplo, consideremos el ejemplo de las manzanas en la cafetería presentado en el documento Chain-of-Thought. En lugar de simplemente llamar al modelo una vez con el indicador de entrada, podemos llamar al modelo tres veces y tomar la respuesta mayoritaria como la respuesta final (ver imagen a continuación).

<img src="../figures/5.png" alt="Principles of Prompt Engineering" width="700"/>


Al adoptar la respuesta de la mayoría, la probabilidad de que el modelo devuelva una salida incorrecta durante una ejecución específica se reduce significativamente. El efecto de esta técnica se potencia al aumentar la repetición. Sin embargo, la ejecución repetida del prompt implica una mayor cantidad de llamadas al modelo, lo que lleva a un mayor consumo de hardware. En el caso de modelos de lenguajes grandes como GPT-3, esto también se traduce en más llamadas a la API y, en consecuencia, más dinero gastado, todo para generar una única respuesta.

Por este motivo, a la hora de diseñar sistemas que utilicen LLM, es fundamental equilibrar la robustez y el ahorro de llamadas al modelo, utilizando esta técnica solo cuando sea estrictamente necesario. Por ejemplo, cuando se necesita un alto nivel de robustez en la respuesta del sistema.

Para ilustrar el potencial de esta técnica, volvamos a nuestra aplicación de Walmart para analizar un caso particular: [huevos](https://www.walmart.com/browse/food/eggs/976759_9176907_1001469?povid=GlobalNav_rWeb_Grocery_DairyEggs_Eggs).



**Products 🛒**

In [14]:
products = f"""
---
Great Value Cage Free Large AA White Eggs, 6 Count

$ 1.07

---
Happy Egg Organic Free Range Large Brown Eggs, 12 Count

$ 5.16

---
Great Value Cage Free Large AA White Eggs, 12 Count

$ 1.86

---
Eggland's Best Farm Fresh Cage Free Large White Grade AA Eggs, 18 Count

$ 2.69

---
Marketside Large Cage Free Brown Eggs, 12 Count

$ 2.67
"""

Agreguemos algunas preferencias de usuario como parte del prompt, para hacer el problema un poquito más desafiante para el modelo.

In [ ]:
preferences = f"""
- Only consider grade AA eggs
- Only consider eggs produced on cage-free farms
- The client needs a minimum of 6 eggs and a maximum of 12
"""

Utilizando modelos poderosos como GPT-3/GPT-4, un prompt Zero-Shot CoT como el siguiente debería ser suficiente para obtener resultados descentes. 

**Prompt 💬**

In [15]:
prompt = f"""
You will be provided with a list of various product presentations delimited by <>. Each product presentation is accompanied by its price. Your task is to identify which presentation offers the best money-saving opportunity, reasoning step-by-step:

Also, consider the following user preferences to select the most suitable ooption:

{preferences}

<{products}>
"""

NameError: name 'preferences' is not defined

**Response 🤖** &nbsp;&nbsp;<span class="label success">GPT-4</span>

```python
(1)
Considering the user preferences, we first filter the list of eggs for only grade AA and cage-free options, which gives us the following choices:

* Great Value Cage Free Large AA White Eggs, 6 Count: $1.07
* Great Value Cage Free Large AA White Eggs, 12 Count: $1.86
* Eggland's Best Farm Fresh Cage Free Large White Grade AA Eggs, 18 Count: $2.69
* The Happy Egg and Marketside options are discarded because their grade is not specified.

Next, we consider the amount of eggs. The client needs a minimum of 6 eggs and a maximum of 12. Therefore, we can discard the Eggland's Best 18 count option, as it exceeds the maximum required amount.

Now, we compare the remaining two options:

* Great Value Cage Free Large AA White Eggs, 6 Count: $1.07
* Great Value Cage Free Large AA White Eggs, 12 Count: $1.86
* Although the 6-count pack fits the minimum requirement, the 12-count pack offers a better price per egg. For the 6-count pack, each egg costs $1.07 / 6 = ~$0.178. For the 12-count pack, each egg costs $1.86 / 12 = ~$0.155.

Therefore, considering both the user's preferences and the cost per egg, the "Great Value Cage Free Large AA White Eggs, 12 Count" for $1.86 is the best money-saving opportunity.

(2)
Let's go through each product one by one, considering the user's preferences:

1. Great Value Cage Free Large AA White Eggs, 6 Count - $1.07
    These are grade AA eggs from a cage-free farm, so they meet the user's criteria. The price per egg is $1.07/6 = $0.178.

2. Happy Egg Organic Free Range Large Brown Eggs, 12 Count - $5.16
    These are not specified as grade AA, so we can't consider them based on user's preferences.

3. Great Value Cage Free Large AA White Eggs, 12 Count - $1.86
    These are grade AA eggs from a cage-free farm and the quantity is within the range the user wants. The price per egg is $1.86/12 = $0.155.

4. Eggland's Best Farm Fresh Cage Free Large White Grade AA Eggs, 18 Count - $2.69
    These are grade AA eggs from a cage-free farm. However, the quantity is more than the user wants (18), so we can't consider this option.

5. Marketside Large Cage Free Brown Eggs, 12 Count - $2.67
    These are not specified as grade AA, so we can't consider them based on user's preferences.

Based on the preferences, we are only left with two choices: Great Value Cage Free Large AA White Eggs, either 6 Count or 12 Count. However, the 12 Count package offers the best money-saving opportunity, because it has a lower cost per egg ($0.155 per egg) than the 6 Count package ($0.178 per egg). Additionally, the 12 Count package still falls within the user's desired quantity range of 6 to 12 eggs. Therefore, the best option for the user is the "Great Value Cage Free Large AA White Eggs, 12 Count" at $1.86.

(3)
Given the user's preferences, we need to find the product which:

1. Contains only grade AA eggs
2. Is produced on cage-free farms
3. Offers a minimum of 6 eggs and a maximum of 12

Let's consider each product:

1. "Great Value Cage Free Large AA White Eggs, 6 Count" - $ 1.07. This option matches all the user's preferences and costs $1.07 for 6 eggs, which is approximately $0.178 per egg.

2. "Happy Egg Organic Free Range Large Brown Eggs, 12 Count" - $ 5.16. This product does not specify the grade of the eggs, so we will disregard this option.

3. "Great Value Cage Free Large AA White Eggs, 12 Count" - $ 1.86. This option also matches the user's preferences and costs $1.86 for 12 eggs, which is approximately $0.155 per egg.

4. "Eggland's Best Farm Fresh Cage Free Large White Grade AA Eggs, 18 Count" - $ 2.69. While this product matches the egg grade and production method preferences, it contains more than the maximum amount of eggs requested by the user (18 instead of 12). Therefore, we disregard this option.

5. "Marketside Large Cage Free Brown Eggs, 12 Count" - $ 2.67. This product does not specify the grade of the eggs, so we will disregard this option.

Therefore, the product that provides the best money-saving opportunity while meeting the user's preferences is "Great Value Cage Free Large AA White Eggs, 12 Count" for $1.86. This option offers more eggs for a lower cost per egg compared to the other suitable option.
```

A pesar de que el modelo no produce resultados incorrectos en ninguna de sus ejecuciones (con un razonamiento preciso en las tres instancias), este ejemplo enfatiza el potencial de variabilidad en las respuestas de un LLM. Además, subraya el valor del método self-consistency en la reducción de dicha variabilidad.

Más en general, se ha demostrado que el método self-consistency mejora los resultados en tareas de razonamiento aritmético, de sentido común y simbólico. Incluso cuando se descubrió que el CoT regular era ineficaz, la autoconsistencia aún podía mejorar los resultados (consulte [resultados experimentales](https://arxiv.org/abs/2203.11171)).

Para concluir esta técnica avanzada, te animo a que la pruebes tú mismo, por ejemplo, agregando más productos a la lista o quitando el “razonamiento paso a paso” para darle al modelo más espacio para cometer errores. Además, intente aplicar la autoconsistencia a los problemas de la vida real en los que está trabajando.

In [16]:
# Now is your turn!
response = get_completion(prompt)
print(response)

A: Presentation 1 costs $11.86 and includes 24 cans, so the price per fluid ounce is $11.86/(24 x 12) = $0.041 per fluid ounce. Presentation 2 costs $7.46 and includes 12 cans, so the price per fluid ounce is $7.46/(12 x 12) = $0.051 per fluid ounce. Presentation 3 costs $4.98 and includes 6 bottles, so the price per fluid ounce is $4.98/(6 x 16.9) = $0.049 per fluid ounce. In conclusion, the cheapest presentation is the first one with a price of $0.041 per fluid ounce.


## References
<a class="anchor" id="references"></a>

* [Prompt Engineering 101: Prompts, Outputs and Notebooks with code]()

* [Wei, J., Wang, X., Schuurmans, D., Bosma, M., Chi, E., Le, Q., & Zhou, D. (2022). Chain of thought prompting elicits reasoning in large language models]()

* [Wang, X., Wei, J., Schuurmans, D., Le, Q., Chi, E., & Zhou, D. (2022). Self-consistency improves chain of thought reasoning in language models]()

* [Brown, T., Mann, B., Ryder, N., Subbiah, M., Kaplan, J. D., Dhariwal, P., ... & Amodei, D. (2020). Language models are few-shot learners]()

* [ChatGPT Prompt Engineering for Developers]() by DeepLearning.ai

* [Building Systems with the ChatGPT API]() by DeepLearning.ai

* [The Full Stack LLM Bootcamp]()

* Prompt Engineering Guide
https://www.promptingguide.ai/

* Introductory Course on Prompting
https://learnprompting.org/